In [10]:
# %load D:/BUDS_MID/__init__.py
"""
Created on Wed Feb 19 16:21:00 2020

@author: cmchico

usual imports
"""

import os
import pandas as pd
import numpy as np

pd.options.display.max_rows = 100
pd.set_option('display.float_format', '{:.0f}'.format)


import f_validate_g4s as ed

from time import time
from datetime import timedelta

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'\nCreated on Wed Feb 19 16:21:00 2020\n\n@author: cmchico\n\nusual imports\n'

In [11]:
# %load D:/BUDS_MID/__initdask__.py
"""
Created on Wed Feb 19 16:24:03 2020

@author: cmchico
"""
import dask.bag as db
from dask.distributed import Client, progress

client = Client(processes = True,n_workers=12)
client

'\nCreated on Wed Feb 19 16:24:03 2020\n\n@author: cmchico\n'

Client Scheduler: tcp://127.0.0.1:58869 Dashboard: http://127.0.0.1:58870/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


In [12]:
path = "D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/"

In [13]:
thresh = pd.read_pickle(path+'89 Threshold Recommendations.pkl')
thresh = thresh[['tid','thresh']].drop_duplicates()
thresh.tid = thresh.tid.astype(str)
thresh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 658 entries, 0 to 230607
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tid     658 non-null    object 
 1   thresh  658 non-null    float64
dtypes: float64(1), object(1)
memory usage: 15.4+ KB


In [14]:
test_dates = pd.date_range(start='01/01/2019',end='12/31/2019')

In [15]:
mapp = pd.read_pickle(path+'Actual/89 TID Br map.pkl')
#211 branches with just one ATM, can be treated as an ATM i/o Branch
mapp[mapp.TID!=mapp.TID_BR].BR_TIDCOUNT.value_counts()

2    256
1    211
3     42
4      8
Name: BR_TIDCOUNT, dtype: int64

In [16]:
reload_df = pd.read_pickle(path+'Actual/80 Replenishment.pkl')
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['Pre-Emptive'] = reload_df['Pre-Emptive'].str.strip().fillna("NORMAL LOADING")
reload_df['date_str'] = reload_df.DATE.astype(str)


reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(str).str.strip().str.replace("","")
select = reload_df['Current Load Amount']==""
reload_df.loc[select,'Current Load Amount'] = "0"
reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(int)

reload_df = reload_df[reload_df.DATE.isin(test_dates)].copy()
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['TID_BR'] = reload_df.TID.map(mapp.set_index('TID').TID_BR.to_dict())

first_reload = reload_df.drop_duplicates(subset=(['TID'])).copy()

reload_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194966 entries, 0 to 194965
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Last Load Amount     194965 non-null  object        
 1   Current Load Amount  194966 non-null  int32         
 2   % Cash Return/Atm    194966 non-null  float64       
 3   Replen               194966 non-null  object        
 4   Pre-Emptive          194966 non-null  object        
 5   _forecasted          194966 non-null  bool          
 6   _cassettefault       194966 non-null  bool          
 7   _bpi                 194966 non-null  bool          
 8   _others              194966 non-null  bool          
 9   reload_type          194966 non-null  object        
 10  TID                  194966 non-null  object        
 11  DATE                 194966 non-null  datetime64[ns]
 12  date_str             194966 non-null  object        
 13  TID_BR        

In [17]:
reload_df.TID_BR.nunique()

658

In [23]:
reload_df.reload_type.value_counts()

FAILURE     103108
FORECAST     76279
OTHERS       15579
Name: reload_type, dtype: int64

In [24]:
preempt = reload_df[reload_df.reload_type.isin(['FAILURE','OTHERS'])][['date_str','TID_BR','reload_type','Current Load Amount']]

In [25]:
df = pd.concat([pd.read_csv(path+'Prophet Forecast/'+i, dtype={'ds':'str','pred_date':'str','tid':'str'}) 
                for i in os.listdir(path+'Prophet Forecast/')],ignore_index=True)
df['pred_date_D'] = pd.to_datetime(df.pred_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3233412 entries, 0 to 3233411
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Unnamed: 0   int64         
 1   ds           object        
 2   y            float64       
 3   yhat         float64       
 4   pred_date    object        
 5   pred_day     object        
 6   se           float64       
 7   tid          object        
 8   pred_date_D  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 222.0+ MB


In [26]:
df.tid.nunique()

658

In [27]:
df.sort_values(['tid','pred_date','ds'], inplace=True)

In [28]:
actual = df[df.pred_day.isin(['day 0','day 1','day 2'])].pivot_table(index=['tid','pred_date','pred_date_D'],columns='pred_day',values='yhat',aggfunc=sum).reset_index()
actual.columns=['tid','pred_date','pred_date_D','yhat0','yhat1','yhat2']
actual[['date_str','y']] = df[df.pred_day=='day 0'][['ds','y']].reset_index(drop=True)

In [29]:
actual.tid.nunique()

658

In [30]:
atms = [atm for atm in actual.tid.unique() if atm in mapp[mapp.BR_TIDCOUNT==1].TID_BR.unique()]
len(atms)

514

In [40]:
def par(atm,thresh,rel_amt):
    
    date_o = first_reload.loc[first_reload['TID_BR'] == atm].reset_index().DATE[0]
    actual_atm = actual[(actual.tid== atm) & (actual.pred_date_D >= date_o-timedelta(days=1))].reset_index(drop=True).copy()

    g4s=preempt[preempt.TID_BR==atm].set_index('date_str')
    g4s['Current Load Amount'] = rel_amt
    g4s['Pre-Emptive'] = g4s.reload_type
    
    threshold = thresh/rel_amt
    if g4s.shape[0]==0: g4s=None
    sched_= ed.sched(actual_atm,rel_amt=rel_amt,pred=pred,r=r,reload_charge=reload_charge,
                      threshold=threshold, hor=hor,g4s=g4s)
    sched_['fix_rel']=rel_amt
    sched_['thresh'] = thresh
    sched_['thresh%'] = threshold
    ss=sched_.groupby('cog_preemptive') \
                        [['reload_flag','cashout_flag','thresh_breach_flag','necessary_flag',
                                             'civ_cost','reload_cost']].sum().reset_index()
    ss['tid']=atm
    ss['fix_rel'] = rel_amt
    ss['thresh'] = thresh
    ss['thresh%'] = threshold
    sched_.to_csv(path+'Prophet Test Thresh R/_sched_'+atm+'_'+str(rel_amt)[:2]+'.csv')
    ss.to_csv(path+'Prophet Test Perf Thresh R/_schedperf_'+ atm+'_'+str(rel_amt)[:2]+'.csv')
    
    return sched_,ss

In [35]:
rel_amt=3200000
pred = ['yhat0','yhat1','yhat2']
hor=len(pred)-1
r = 0.05/365
reload_charge=2000
# threshold=0.25

In [36]:
#3atms per minute
# rel_amt = 3200000
# start=time()
# for atm in atms[:10]:
#     print(atm)
#     thresh_ = thresh[thresh.tid==atm].reset_index(drop=True).thresh[0]
#     a,b= par(atm,thresh_,rel_amt)
#     print(timedelta(seconds=time()-start))

In [45]:
atm

'91010033'

In [41]:
atm=atms[0]
thresh_ = thresh[thresh.tid==atm].reset_index(drop=True).thresh[0]
a=par(atm,thresh_,3200000)
# # par(atm,thresh_,4200000)
# # par(atm,thresh_,5200000)
# # par(atm,thresh_,6200000)

   start_bal  start_reload  total_bal  end_bal   yhat
0    3199000             0    3199000  3014332 184668
1    3014332             0    3014332  2848288 166044
2    2848288             0    2848288  2494710 353578
3    2494710           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3198500             0    3198500  3034938 163562
1    3034938             0    3034938  2681779 353159
2    2681779             0    2681779  2338572 343207
3    2338572           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3185400             0    3185400  2833819 351581
1    2833819             0    2833819  2491425 342394
2    2491425             0    2491425  2422679  68746
3    2422679           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3056400             0    3056400  2717255 339145
1    2717255             0    2717255  2649588  67668
2    2649588             0  

3    2711505           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0          0       3200000    3200000  2901208 298792
1    2901208       3200000    3200000  2997311 202689
2    2997311             0    2997311  2759469 237842
3    2759469           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3139700       3200000    3200000  3000212 199788
1    3000212             0    3000212  2761192 239020
2    2761192       3200000    3200000  2971235 228765
3    2971235           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2962500             0    2962500  2725717 236783
1    2725717       3200000    3200000  2975449 224551
2    2975449             0    2975449  2826059 149390
3    2826059           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2784900       3200000    3200000  2975626 224374
1    2975626             0  

   start_bal  start_reload  total_bal  end_bal   yhat
0    3024800             0    3024800  2619415 405385
1    2619415             0    2619415  2376053 243362
2    2376053             0    2376053  2087285 288768
3    2087285           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2926700             0    2926700  2688189 238511
1    2688189             0    2688189  2403257 284933
2    2403257             0    2403257  2096890 306366
3    2096890           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2859000             0    2859000  2577635 281365
1    2577635             0    2577635  2273615 304020
2    2273615             0    2273615  2273615      0
3    2273615           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2746300             0    2746300  2446711 299589
1    2446711             0    2446711  2446711      0
2    2446711             0  

   start_bal  start_reload  total_bal  end_bal    yhat
0    2759700             0    2759700  2542712  216988
1    2542712             0    2542712  2156524  386188
2    2156524             0    2156524   144820 2011704
3     144820           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    2759700             0    2759700  2382735  376965
1    2382735       3200000    3200000  1191890 2008110
2    1191890       3200000    3200000  2638785  561215
3    2638785           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    2140700       3200000    3200000  1199598 2000402
1    1199598       3200000    3200000  2629844  570156
2    2629844             0    2629844  2163527  466317
3    2163527           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal   yhat
0     685700       3200000    3200000  2629538 570462
1    2629538             0    2629538  2168513 461025
2    2168513 

   start_bal  start_reload  total_bal  end_bal   yhat
0     868500             0     868500   255744 612756
1     255744       3200000    3200000  2851044 348956
2    2851044             0    2851044  2584302 266742
3    2584302           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0     297500       3200000    3200000  2851613 348387
1    2851613       3200000    3200000  2933811 266189
2    2933811       3200000    3200000  2996156 203844
3    2996156           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3166700       3200000    3200000  2943901 256099
1    2943901       3200000    3200000  3000121 199879
2    3000121             0    3000121  2895250 104871
3    2895250           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3022500       3200000    3200000  3002618 197382
1    3002618             0    3002618  2901487 101131
2    2901487       3200000  

3     658199           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    3200000             0    3200000  1246718 1953282
1    1246718             0    1246718   847183  399535
2     847183             0     847183   631130  216053
3     631130           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3200000             0    3200000  2828803 371197
1    2828803             0    2828803  2629348 199455
2    2629348             0    2629348  2507175 122173
3    2507175           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2928800             0    2928800  2731660 197140
1    2731660             0    2731660  2611719 119941
2    2611719             0    2611719  2499618 112101
3    2499618           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2761200             0    2761200  2643961 117239
1    2643961          

   start_bal  start_reload  total_bal  end_bal  yhat
0    2483100             0    2483100  2483100     0
1    2483100             0    2483100  2483100     0
2    2483100             0    2483100  2423797 59303
3    2423797           nan        nan      nan   nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2479300             0    2479300  2479300      0
1    2479300             0    2479300  2413672  65628
2    2413672             0    2413672  2305789 107883
3    2305789           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2395600             0    2395600  2327363  68237
1    2327363             0    2327363  2218241 109122
2    2218241             0    2218241  2116992 101250
3    2116992           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    2317300             0    2317300  2208490  108810
1    2208490             0    2208490  2106947  101543
2    2106947       3200000    

   start_bal  start_reload  total_bal  end_bal   yhat
0    2415700             0    2415700  2356216  59484
1    2356216       3200000    3200000  3069952 130048
2    3069952             0    3069952  2863639 206313
3    2863639           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2314500       3200000    3200000  3068766 131234
1    3068766             0    3068766  2860306 208460
2    2860306             0    2860306  2618983 241324
3    2618983           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3055900             0    3055900  2847851 208049
1    2847851             0    2847851  2605028 242824
2    2605028             0    2605028  2597299   7729
3    2597299           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2879600             0    2879600  2638284 241316
1    2638284             0    2638284  2628361   9923
2    2628361       3200000  

   start_bal  start_reload  total_bal  end_bal   yhat
0    2629500             0    2629500  2594543  34957
1    2594543             0    2594543  2498748  95795
2    2498748             0    2498748  2334501 164246
3    2334501           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2556000             0    2556000  2461844  94156
1    2461844             0    2461844  2298731 163113
2    2298731             0    2298731  2298731      0
3    2298731           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2387000             0    2387000  2222832 164168
1    2222832             0    2222832  2222832      0
2    2222832             0    2222832  2222832      0
3    2222832           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal  yhat
0    2353000             0    2353000  2353000     0
1    2353000             0    2353000  2353000     0
2    2353000             0    2

   start_bal  start_reload  total_bal  end_bal   yhat
0    3056000             0    3056000  2917136 138864
1    2917136             0    2917136  2878694  38442
2    2878694             0    2878694  2751005 127689
3    2751005           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2915000             0    2915000  2874104  40896
1    2874104             0    2874104  2744675 129428
2    2744675             0    2744675  2184798 559877
3    2184798           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2750500             0    2750500  2549746 200754
1    2549746             0    2549746  1981545 568200
2    1981545             0    1981545  1981545      0
3    1981545           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal  yhat
0    2568000             0    2568000  2490227 77773
1    2490227             0    2490227  2490227     0
2    2490227             0    2

In [44]:
a[1]

,cog_preemptive,reload_flag,cashout_flag,thresh_breach_flag,necessary_flag,civ_cost,reload_cost,tid,fix_rel,thresh,thresh%
0,FAILURE,23,0,0,0,8714,46000,91010033,3200000,600000,0
1,No Reload,31,4,4,0,103019,62000,91010033,3200000,600000,0
2,OTHERS,23,0,0,0,9286,46000,91010033,3200000,600000,0


In [46]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=3200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:15:55.744565


In [47]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=4200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:15:19.141380


In [48]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=5200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:14:47.461571


In [49]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=6200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:14:38.574799


In [ ]:
perf = pd.concat(pd.re)

In [55]:
perf = pd.concat(pd.read_csv(path+'Prophet Test Perf Thresh R/' + i).assign(file=i)
          for i in os.listdir(path+'Prophet Test Perf Thresh R')[2:])

In [117]:
perfs = perf.groupby(['tid','file'])[['civ_cost','reload_cost','cashout_flag','reload_flag']].sum().reset_index()
perfs['total_cost'] = perfs.civ_cost+perfs.reload_cost
perfs['reload_amt'] = perfs.file.str.split('_').str[3].str.split(".").str[0].astype(int)*100000

In [118]:
mcost = perfs.sort_values(['total_cost','cashout_flag']).drop_duplicates(subset='tid')

In [119]:
mcost.sum()
mcost.mean()

file            _schedperf_91010068_32.csv_schedperf_91010383_...
civ_cost                                                 78358445
reload_cost                                             130648000
cashout_flag                                                 1447
reload_flag                                                 65324
total_cost                                              209006445
reload_amt                                             2612800000
dtype: object

civ_cost        152448
reload_cost     254179
cashout_flag         3
reload_flag        127
total_cost      406627
reload_amt     5083268
dtype: float64

In [120]:
mcashout = perfs.sort_values(['cashout_flag','total_cost']).drop_duplicates(subset='tid')

In [121]:
mcashout.sum()
mcashout.mean()

file            _schedperf_91010068_32.csv_schedperf_91010383_...
civ_cost                                                 80837994
reload_cost                                             148896000
cashout_flag                                                  546
reload_flag                                                 74448
total_cost                                              229733994
reload_amt                                             2634800000
dtype: object

civ_cost        157272
reload_cost     289681
cashout_flag         1
reload_flag        145
total_cost      446953
reload_amt     5126070
dtype: float64

In [103]:
g4s = pd.read_csv(path+'85 G4S perf 2019.csv', dtype={'tid':'str'}).merge(mapp,left_on='tid',right_on='TID')

In [114]:
g4ss = g4s[g4s.TID_BR.isin(atms)].groupby('TID_BR')[['civ_cost','reload_cost','cashout_flag','reload_flag']].sum().reset_index()
g4ss['total_cost'] = g4ss.civ_cost + g4ss.reload_cost

In [115]:
g4ss.TID_BR.nunique()

514

In [116]:
g4ss.sum()
g4ss.mean()

TID_BR          9101003391010053910100689101007591010090910101...
civ_cost                                                 81881837
reload_cost                                             143456000
cashout_flag                                                 7666
reload_flag                                                 71728
total_cost                                              225337837
dtype: object

civ_cost       159303
reload_cost    279097
cashout_flag       15
reload_flag       140
total_cost     438400
dtype: float64

In [123]:
225337837-209006445

16331392

In [20]:
# len(atms)

514

In [21]:
# generated = pd.Series(os.listdir(path+'/Prophet Test Thresh R/OFFSITE OR ONE ATM BRANCH')).str.split('_').str[2].unique()

In [22]:
# len(generated)

393

In [23]:
# atms_new = list(set(atms) - set(generated))

In [24]:
# len(atms_new)

121

In [26]:
# dbatms = db.from_sequence(atms_new)
# dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms_new])
# dbmaster = dbatms.map(par,dbthresh,rel_amt=3200000)

# start=time()
# df = dbmaster.compute()
# print(timedelta(seconds=time()-start))

0:05:45.500126


In [27]:
# dbatms = db.from_sequence(atms_new)
# dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms_new])
# dbmaster = dbatms.map(par,dbthresh,rel_amt=4200000)

# start=time()
# df = dbmaster.compute()
# print(timedelta(seconds=time()-start))

0:05:46.223067


In [28]:
# dbatms = db.from_sequence(atms_new)
# dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms_new])
# dbmaster = dbatms.map(par,dbthresh,rel_amt=5200000)

# start=time()
# df = dbmaster.compute()
# print(timedelta(seconds=time()-start))

0:05:35.269140


In [29]:
# dbatms = db.from_sequence(atms_new)
# dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms_new])
# dbmaster = dbatms.map(par,dbthresh,rel_amt=6200000)

# start=time()
# df = dbmaster.compute()
# print(timedelta(seconds=time()-start))

0:05:13.991721
